# Testing all non-MCMC functions

In [2]:
import numpy as np
from pyMuellerMat import MuellerMat
from pyMuellerMat import common_mms as cmm
from pseudo_code_from_Max import generate_system_mueller_matrix, \
    update_systemMM, generate_measurement, parse_configuration


# Define the system dictionary
# NOTE: Retardances are in radians, thetas are in degrees
system_dict = {
    "components": {
        "parang_rot": {
            "type": "Rotator",
            "properties": {"pa": 0},
            "tag": "on-sky",
        },
        "m3": {
            "type": "DiattenuatorRetarder",
            "properties": {"phi": 2 * np.pi * 0, "epsilon": 0.03},
            "tag": "on-sky",
        },
        "alt_rot": {
            "type": "Rotator",
            "properties": {"pa": 45},
            "tag": "internal",
        },
        "hwp": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0.5, "theta": 45, "delta_theta": 1},
            "tag": "internal",
        },
        "image_rotator": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0.5, "theta": 0, "delta_theta": 1},
            "tag": "internal",
        },
        "flc": {
            "type": "Retarder",
            "properties": {"phi": 2 * np.pi * 0.5, "theta": 45},
            "tag": "internal",
        },
        "wollaston": {
            "type": "WollastonPrism",
            "properties": {"beam": "o", "transmission_ratio": 0.5},
            "tag": "internal",
        },
    },
    # NOTE: This needs to be from downstream to upstream
     "order": [
        "wollaston", "flc", "image_rotator", "hwp", "alt_rot", "m3", "parang_rot"
    ],
}

# Test `parse_configuration`
values, keywords = parse_configuration(system_dict["components"])
print("Parsed configuration values:", values)
print("Parsed configuration keywords:", keywords)

# Generate the system Mueller matrix
system_mm = generate_system_mueller_matrix(system_dict)

# Print the original evaluated system Mueller matrix
print("Original System Mueller Matrix:")
original_matrix = system_mm.evaluate()  # Evaluate to compute the full matrix
print(original_matrix)

# Test `generate_measurement`
S_in = [1, 0, 0, 0]  # Unpolarized light
S_out = generate_measurement(system_mm, S_in)
print("Unpolarized Stokes vector after passing through the system:")
print(S_out)

# Test `update_systemMM`
parameter_updates = {("hwp", "theta"): 10, ("image_rotator", "theta"): 5}
system_parameters = [["hwp", "theta"], ["image_rotator", "theta"]]

# Update the parameters and evaluate again
update_systemMM(parameter_updates, system_mm, system_parameters)
print("Updated System Mueller Matrix:")
updated_matrix = system_mm.evaluate()  # Evaluate again after updates
print(updated_matrix)


Parsed configuration values: ['Rotator', {'pa': 0}, 'on-sky', 'DiattenuatorRetarder', {'phi': 0.0, 'epsilon': 0.03}, 'on-sky', 'Rotator', {'pa': 45}, 'internal', 'Retarder', {'phi': 3.141592653589793, 'theta': 45, 'delta_theta': 1}, 'internal', 'Retarder', {'phi': 3.141592653589793, 'theta': 0, 'delta_theta': 1}, 'internal', 'Retarder', {'phi': 3.141592653589793, 'theta': 45}, 'internal', 'WollastonPrism', {'beam': 'o', 'transmission_ratio': 0.5}, 'internal']
Parsed configuration keywords: [['parang_rot', 'type'], ['parang_rot', 'properties'], ['parang_rot', 'tag'], ['m3', 'type'], ['m3', 'properties'], ['m3', 'tag'], ['alt_rot', 'type'], ['alt_rot', 'properties'], ['alt_rot', 'tag'], ['hwp', 'type'], ['hwp', 'properties'], ['hwp', 'tag'], ['image_rotator', 'type'], ['image_rotator', 'properties'], ['image_rotator', 'tag'], ['flc', 'type'], ['flc', 'properties'], ['flc', 'tag'], ['wollaston', 'type'], ['wollaston', 'properties'], ['wollaston', 'tag']]
Original System Mueller Matrix:
[[